# Notebook to generate indicators module

In [120]:
from pathlib import Path
from pprint import pformat

from inspect import Signature, Parameter

from mintalib import core
from mintalib.samples import sample_prices

ROOTDIR = Path.cwd().parent
PKGDIR = ROOTDIR.joinpath("src/mintalib").resolve(strict=True)

%load_ext nbmask

In [ ]:
prices = sample_prices()
prices.info()


In [122]:


import importlib.util

def new_module(name: str, code: str = None):
    spec = importlib.util.spec_from_loader(name, None)
    module = importlib.util.module_from_spec(spec)
    if code:
        exec(code, module.__dict__)
    return module

In [ ]:
def core_functions():
    return tuple(k for k, v in vars(core).items() if k.isupper() and callable(v))

core_functions()

In [124]:
# Sample code for indicator

from mintalib.model import Indicator
from mintalib.core import wrap_indicator

@wrap_indicator(core.EMA)
class EMA(Indicator):
    same_scale = True

    def __init__(self, period: int, *, item: str = None):
        self.period = period
        self.item = item

    def __call__(self, series):
        return core.EMA(series, period=self.period)

In [ ]:
PREAMBLE = '''
""" Mintalib Indicators """

# Do not edit! This file was generated by make-indicators.ipynb

from . import core
from . import model

nan = float('nan')

'''


def make_indicator(func, name: str = None, preamble=False, verbose=False):
    if name is None:
        name = func.__name__.upper()

    fname = func.__qualname__
    module = func.__module__.rpartition(".")[2]
    qname = f"{module}.{fname}"

    indent = " " * 4
    indent2 = " " * 8

    signature = Signature.from_callable(func)
    parameters = list(signature.parameters.values())

    if verbose:
        print("signature", signature)
        print("parameters", parameters)

    newparams = []
    ftype = parameters[0].name

    def annotate(param):
        if param.name in ["period"]:
            return param.replace(annotation=int)

        if param.annotation == "str":
            return param.replace(annotation=str)

        if type(p.default) in (int, float, bool):
            return param.replace(annotation=type(p.default))

        return param

    for i, p in enumerate(parameters):
        if i == 0:
            p = Parameter("self", Parameter.POSITIONAL_OR_KEYWORD)
        else:
            p = annotate(p)
        newparams.append(p)

    if verbose:
        print("newparams", newparams)

    newsig = Signature(newparams)

    def argument(arg):
        return ftype if arg == "self" else f"{arg}=self.{arg}"

    args = [argument(p.name) for p in newparams]
    args = ", ".join(args)

    code = PREAMBLE if preamble else ""
    # code = "# noinspection PyPep8Naming\n"

    code += f"@core.wrap_function({qname})\n"
    code += f"class {name}(model.Indicator):\n"

    if len(newparams) > 1:
        code += indent + f"def __init__{newsig}:\n"
        for p in newparams[1:]:
            name = p.name
            code += indent2 + f"self.{name} = {name}\n"
        code += "\n"

    code += indent + f"def __call__(self, {ftype}):\n"
    code += indent2 + f"return {qname}({args})\n"

    return code


print(make_indicator(core.EMA))

In [ ]:
def test_indicator(func, name=None, verbose=False):
    code = make_indicator(func, name=name, preamble=True, verbose=verbose)
    print(code)
    return new_module("mintalib.functions", code)


test_indicator(core.EMA)

In [ ]:
def make_indicators(names=None):
    if names is None:
        names = core_functions()

    output = PREAMBLE

    buffer = pformat(names, width=75, compact=True, indent=4)
    buffer = buffer.replace("(", " ").replace(")", "")
    output += f"__all__ = [\n{buffer}\n]\n"

    output += "\n\n"

    for name in names:
        func = getattr(core, name)
        text = make_indicator(func)
        output += text + "\n\n"


    return output


code = make_indicators()
print(code[:256], "...")

In [ ]:
code = make_indicators()

indicators = new_module("mintalib.indicators", code)

dir(indicators)

In [129]:
assert callable(indicators.SMA)
assert callable(indicators.SMA(20))

res = indicators.SMA(20) @ prices

assert len(res) > 0


In [ ]:
outfile = PKGDIR / "indicators.py"

print(f"Updating {outfile.name} ...")

outfile.write_text(code)

In [131]:
from mintalib import indicators

In [ ]:
help(indicators)